In [2]:
import argilla as rg
import json
import os
import time
from datasets import load_dataset, load_from_disk, Dataset, load_metric, concatenate_datasets
import pandas as pd
import openai
from datetime import datetime
import pickle
openai.api_key = os.environ["OPENAI_API_KEY"]

/Users/pablo/.local/share/virtualenvs/finadvice-dataset-mdWLTC9R/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def complete_chatgpt(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1024, sleep_seconds=1.2):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
        )

        if sleep_seconds > 0:
            time.sleep(sleep_seconds)

        return response["choices"][0]["message"]["content"].strip()

    except Exception as e:
        print(e)
        return ""

"""
possible roles: system, assistant, user
"""
def create_chatgpt_message(role: str, content: str):
     return {"role": role, "content": content}

In [4]:
system_message = create_chatgpt_message("system", "You are a financial advisor AI that aims to provide support for personal finance-related issues.")
def wrap_finadv_submission_call(text):
    user_message = create_chatgpt_message("user", text)
    messages = [system_message, user_message]

    return complete_chatgpt(messages, model="gpt-4", temperature=0.7)

In [9]:
# messages = [create_chatgpt_message("user", "what is the square root of 33?")]
# complete_chatgpt(messages, model="gpt-4")

In [5]:
df = pd.read_csv("./outdata/submissions.csv")
assert(len(df["id"].unique()) == len(df))
display(df.sample())
len(df)

,id,title,created_utc,edited,selftext
24270,wyo148,Can I adjust my W4 for my job in order to rece...,1661558036,1.661562e+09,EDIT: Thank you so much for your replies. I un...


84944

In [12]:
df.head()

,id,title,created_utc,edited,selftext
0,1006ury,Are financial advisors mainly for people who a...,1672530374,-1.0,I grew up as an only child with my dad working...
1,1005xmw,Looking for Thoughts from Strangers,1672527497,-1.0,"Couple w/ Gross Income ~$127,000\n\nTake Home ..."
2,1005hzw,Budgeting buying a vehicle for the first time,1672526170,-1.0,I'm in the military and am moving to Californi...
3,1005gb0,Questions about T-bill auction day,1672526027,-1.0,"&amp;#x200B;\n\n \nFor t-bill 912796YN3, anno..."
4,1005azk,SLS Mortgage Pulled funds on the 30th of this ...,1672525594,-1.0,I woke up yesterday surprised that *Specialize...


In [18]:
# DO NOT OVERWRITE THIS ON A RUNNING KERNEL TO AVOID DELETING CHATGPT ANSWERS THAT HAVENT BEEN SAVED TO DISK YET
# id_answer_map = {}

In [10]:
def retrieve_chatgpt_answers_to_submissions(limit=10):
    limit_counter = 0

    for idx, row in df.iterrows():
        if limit_counter >= limit:
            print(f"limit of {limit} reached")
            break

        id = row["id"]
        if id in id_answer_map:
            continue

        print(f"[{id}]🏃‍♂️ retrieving answer...")
        question_text = row["selftext"]
        answer_text = wrap_finadv_submission_call(question_text)
        if answer_text == "":
            print("chatgpt seems to have errored out, aborting")
            return

        print(f"[{id}]✅ retrieved answer")
        id_answer_map[id] = answer_text
        
        limit_counter += 1

In [11]:
retrieve_chatgpt_answers_to_submissions(50)

[1000nwb]🏃‍♂️ retrieving answer...
[1000nwb]✅ retrieved answer
[1000joe]🏃‍♂️ retrieving answer...
[1000joe]✅ retrieved answer
[1000hsv]🏃‍♂️ retrieving answer...
[1000hsv]✅ retrieved answer
[1000eqi]🏃‍♂️ retrieving answer...
[1000eqi]✅ retrieved answer
[1000cmu]🏃‍♂️ retrieving answer...
[1000cmu]✅ retrieved answer
[1000c3c]🏃‍♂️ retrieving answer...
[1000c3c]✅ retrieved answer
[1000bax]🏃‍♂️ retrieving answer...
[1000bax]✅ retrieved answer
[1000997]🏃‍♂️ retrieving answer...
[1000997]✅ retrieved answer
[100053n]🏃‍♂️ retrieving answer...
[100053n]✅ retrieved answer
[10004nl]🏃‍♂️ retrieving answer...
[10004nl]✅ retrieved answer
[zzzo10]🏃‍♂️ retrieving answer...
[zzzo10]✅ retrieved answer
[zzzm70]🏃‍♂️ retrieving answer...
[zzzm70]✅ retrieved answer
[zzz9sg]🏃‍♂️ retrieving answer...
[zzz9sg]✅ retrieved answer
[zzz8wz]🏃‍♂️ retrieving answer...
[zzz8wz]✅ retrieved answer
[zzyo7a]🏃‍♂️ retrieving answer...
[zzyo7a]✅ retrieved answer
[zzyjbd]🏃‍♂️ retrieving answer...
[zzyjbd]✅ retrieved answer
[zzy

In [38]:
retrieve_chatgpt_answers_to_submissions(40)

[1004q1u]🏃‍♂️ retrieving answer...
[1004q1u]✅ retrieved answer
[1004k51]🏃‍♂️ retrieving answer...
[1004k51]✅ retrieved answer
[1004isk]🏃‍♂️ retrieving answer...
[1004isk]✅ retrieved answer
[1004djv]🏃‍♂️ retrieving answer...
[1004djv]✅ retrieved answer
[1004cue]🏃‍♂️ retrieving answer...
[1004cue]✅ retrieved answer
[10049g4]🏃‍♂️ retrieving answer...
[10049g4]✅ retrieved answer
[10048b4]🏃‍♂️ retrieving answer...
[10048b4]✅ retrieved answer
[10043f4]🏃‍♂️ retrieving answer...
[10043f4]✅ retrieved answer
[1003u24]🏃‍♂️ retrieving answer...
[1003u24]✅ retrieved answer
[1003rme]🏃‍♂️ retrieving answer...
[1003rme]✅ retrieved answer
[1003on7]🏃‍♂️ retrieving answer...
[1003on7]✅ retrieved answer
[1003o86]🏃‍♂️ retrieving answer...
[1003o86]✅ retrieved answer
[1003c7h]🏃‍♂️ retrieving answer...
[1003c7h]✅ retrieved answer
[1003955]🏃‍♂️ retrieving answer...
[1003955]✅ retrieved answer
[1002pwp]🏃‍♂️ retrieving answer...
[1002pwp]✅ retrieved answer
[1002h3v]🏃‍♂️ retrieving answer...
[1002h3v]✅ retrieved

In [15]:
len(id_answer_map)

100

In [20]:
new_records = []
for idx, row in df.iterrows():
    id = row["id"]
    if not id in id_answer_map:
        break

    question_text = row["selftext"]
    answer_text = id_answer_map[id]

    text = f"### Human\n{question_text}\n\n### Assistant\n{answer_text}<eos_token>"

    new_records.append({"id": id, "question": question_text, "answer": answer_text, "text": text})

qa_df = pd.DataFrame(new_records)
qa_df.to_csv("./outdata/finadv50.csv", index=False)

In [21]:
qa_df.sample()

,id,question,answer,text
16,10048b4,"Hi everyone, I have been having some trouble l...",Hello! It's great to see your motivation and e...,"### Human\nHi everyone, I have been having som..."


In [22]:
ds = Dataset.from_pandas(qa_df)
ds.push_to_hub("nihiluis/finadv100_v2", private=True)

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:05<00:00,  5.40s/it]


In [13]:
now = datetime.now()
now_str = now.strftime("%m-%d-%Y_%Hh%Mm%Ss")

with open(f"./outdata/id_answer_map-{now_str}.pickle", "wb") as handle:
    pickle.dump(id_answer_map, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [8]:
with open("./outdata/id_answer_map-08-22-2023_12h02m53s.pickle", "rb") as handle:
    id_answer_map = pickle.load(handle)
    print(len(id_answer_map))
    #display(test_map)

50
